# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import gmaps
# Import API key
from api_keys import geoapify_api_key
print("geoapify_key :"+geoapify_api_key)

geoapify_key :70afb867fed7d1fb40717aaa968bd33c


In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,78.33,76,100,12.10,PF,1682064780
1,1,yubari,43.0378,141.9578,48.49,52,60,17.72,JP,1682064877
2,2,asau,46.4333,26.4000,51.28,80,100,6.42,RO,1682064836
3,3,ushuaia,-54.8000,-68.3000,38.86,81,75,27.63,AR,1682064686
4,4,bauta,22.9828,-82.5464,67.98,88,0,2.30,CU,1682064878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      #geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,80,qazvin,36.2797,50.0049,74.82,31,0,6.91,IR,1682064898
110,110,pisco,-13.7000,-76.2167,70.02,83,0,4.61,PE,1682064823
241,241,jacksonville,30.3322,-81.6556,72.00,86,0,3.44,US,1682064942
257,257,nushki,29.5542,66.0215,77.54,6,0,8.81,PK,1682064946
271,271,sakakah,29.9697,40.2064,79.27,31,0,3.98,SA,1682064696
427,427,iberia,29.9502,-91.7507,71.96,83,0,6.91,US,1682064995
469,469,esperance,-33.8667,121.9000,73.18,41,0,3.18,AU,1682064777
548,548,mazatlan,23.2167,-106.4167,71.91,93,0,1.99,MX,1682065031


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
80,qazvin,IR,36.2797,50.0049,
110,pisco,PE,-13.7000,-76.2167,
241,jacksonville,US,30.3322,-81.6556,
257,nushki,PK,29.5542,66.0215,
271,sakakah,SA,29.9697,40.2064,
427,iberia,US,29.9502,-91.7507,
469,esperance,AU,-33.8667,121.9000,
548,mazatlan,MX,23.2167,-106.4167,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_api_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
qazvin - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
jacksonville - nearest hotel: No hotel found
nushki - nearest hotel: No hotel found
sakakah - nearest hotel: No hotel found
iberia - nearest hotel: No hotel found
esperance - nearest hotel: No hotel found
mazatlan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
80,qazvin,IR,36.2797,50.0049,No hotel found
110,pisco,PE,-13.7000,-76.2167,No hotel found
241,jacksonville,US,30.3322,-81.6556,No hotel found
257,nushki,PK,29.5542,66.0215,No hotel found
271,sakakah,SA,29.9697,40.2064,No hotel found
427,iberia,US,29.9502,-91.7507,No hotel found
469,esperance,AU,-33.8667,121.9000,No hotel found
548,mazatlan,MX,23.2167,-106.4167,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

 

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   #geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )


 

# Display the map
hotel_map